# **Tutorial** - Dynamic Topic Modeling with Trump's Tweets
(last updated 26-04-2021)

In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Trump's Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [7]:
# !pip install bertopic

In [8]:
# !pip install tbb

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this tutorial, we will be needing to extract all Trump's Tweet from his @realDonalTrump account. We will be removing all retweet and focus on his original tweets. 

Moreover, since we are looking at his tweets over time, we will be saving all timestamps related to his tweets.

In [20]:
import re
import pandas as pd
# using bolt google play store manual annotation review

url='https://drive.google.com/file/d/1qWuyf3UrpaU5xnxLmO3GMFa6zybSFYQh/view?usp=sharing' # old
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url2, usecols=['content','at'])

# remove emoji
# https://stackoverflow.com/a/57514515/2670476
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

df.content = df.apply(lambda row: re.sub(r"http\S+", "", row.content).lower(), 1)
df.content = df.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.content.split())), 1)
df.content = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.content).split()), 1)
tweets = df.content.tolist()
timestamps = df['at'].tolist()
# tweets
# timestamps

In [21]:
# import re
# import pandas as pd
# # using bolt google play store manual annotation review

# # url='https://drive.google.com/file/d/1L6bJ_mdqoMBfoENoqTOMGG1m7vZauijV/view?usp=sharing' # all p3
# url='https://drive.google.com/file/d/1D65Fc5i6khcqrcJjdC8pOUDc_dy3a3C4/view?usp=sharing' # bolt p3
# url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
# df = pd.read_csv(url2, usecols=['review','date'])

# # remove emoji
# # https://stackoverflow.com/a/57514515/2670476
# df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

# df.review = df.apply(lambda row: re.sub(r"http\S+", "", row.review).lower(), 1)
# df.review = df.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.review.split())), 1)
# df.review = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.review).split()), 1)
# tweets = df.review.tolist()
# timestamps = df['date'].tolist()
# # tweets
# # timestamps

In [22]:
tweets[1]

'your gps setting around cape town international airport needs to be looked at it always take my location to the drop off and go section of the airport when i m actually about km away from that spot sometimes the drivers mobile numbers does not match their profile but other than that the app is quite user friendly'

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [23]:
from bertopic import BERTopic
# topic_model = BERTopic(min_topic_size=35, verbose=True)
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(tweets)

Batches:   0%|          | 0/1623 [00:00<?, ?it/s]

2021-07-06 14:43:37,764 - BERTopic - Transformed documents to Embeddings
2021-07-06 14:44:10,122 - BERTopic - Reduced dimensionality with UMAP
2021-07-06 14:44:14,553 - BERTopic - Clustered UMAP embeddings with HDBSCAN


KeyboardInterrupt: 

We can then extract most frequent topics:

In [ ]:
freq = topic_model.get_topic_info()
freq.head(10)

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
# topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_nr = freq.iloc[2]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)   # You can select a topic number as shown above

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [ ]:
fig = topic_model.visualize_topics(); fig

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `topics`
  * The topics that we have created before
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n=20)